In [1]:
# Learn plan:
# - Character RNN
# - Stateless RNN
# - Stateful RNN

In [20]:
import keras.utils
import numpy as np
from keras.src.legacy.preprocessing.text import Tokenizer # TODO: seems this a deprecated class, find out a new way to do this
import tensorflow as tf

In [2]:
shakespear_url = 'https://github.com/karpathy/char-rnn'

file_path = keras.utils.get_file('shakespear.txt', shakespear_url)

with open(file_path) as f:
    shakespear_text = f.read()

 212992/Unknown 0s 1us/step

In [4]:

tokenizer = Tokenizer(char_level=True)  # coding at char level
tokenizer.fit_on_texts(shakespear_text) # fit tokenizer on text

In [5]:
tokenizer.texts_to_sequences(['First'])

[[37, 7, 14, 9, 2]]

In [12]:
tokenizer.sequences_to_texts([[37, 7, 14, 9, 2]])

['f i r s t']

In [14]:
max_id = len(tokenizer.word_index)
max_id # total unique characters in the text

74

In [15]:
data_size = tokenizer.document_count
data_size # total characters in the text

309117

In [18]:
# let's encode the entire text
[encoded] = np.array(tokenizer.texts_to_sequences([
    shakespear_text
])) - 1  # shift to zero-based
encoded

array([37, 37, 37, ..., 29, 37, 37], shape=(309117,))

In [19]:
# Data Split for Training / Validation
train_size = data_size * 90 // 100
train_size

278205

In [21]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [22]:
# let's appy 'truncated backpropagation through time' (TBPTT) method

n_steps = 100 # length of the input sequence
window_length = n_steps + 1 # target - is input shifted by one character
dataset = dataset.window(window_length,       # all windows have the same length
                         shift=1,             # windows are shifted by one character
                         drop_remainder=True) # only full windows